In [ ]:
import sqlite3
import pandas as pd
import multiprocessing
from multiprocessing import Pool
import scipy
import requests
import os

# Data

In [2]:
#Establish connection with our database
dat = sqlite3.connect('/home/jovyan/github/models/research/gan/20170509-bam-2.2m-Nja9G.sqlite') #connected to database with out error

#Querys 
dfSOURCE = pd.read_sql_query("SELECT * FROM modules", dat)
dfSCORES = pd.read_sql_query("SELECT * FROM scores", dat)

#Categorial values we will store, paintType emotion and content
paintType=['media_oilpaint','media_3d_graphics','media_pen_ink','media_comic'
           ,'media_vectorart','media_graphite','media_watercolor']

emotion=['emotion_gloomy','emotion_happy','emotion_peaceful','emotion_scary']
content=['content_bicycle','content_bird','content_building','content_cars','content_cat','content_dog','content_flower','content_people','content_tree']

#Store the categorical value that defines each img

dfSCORES['paintType']=dfSCORES[paintType].idxmax(axis=1)
dfSCORES['emotion']=dfSCORES[emotion].idxmax(axis=1)
dfSCORES['content']=dfSCORES[content].idxmax(axis=1)

dfSCORES['TOPpaintType']= dfSCORES[paintType].max(axis=1)
dfSCORES['TOPemotion']=dfSCORES[emotion].max(axis=1)
dfSCORES['TOPcontent']=dfSCORES[content].max(axis=1)


listC=['mid','paintType','emotion','content','TOPcontent','TOPemotion','TOPpaintType']

In [3]:
df=pd.merge(dfSOURCE, dfSCORES[listC], on =['mid'])
df=df.loc[df['TOPpaintType'] >= -0.15]
df = df.reset_index(drop=True)
listC=['mid','src','paintType','emotion','content']
df=df[listC]
df = df.reset_index(drop=True)

In [4]:
#df = df.loc[df['paintType'] == 'media_watercolor']
df = df.reset_index(drop=True)

In [ ]:

for type in paintType:
    if not os.path.exists(type):
        os.mkdir(type)

path = '/home/jovyan/github/models/BAM-DCGAN/data/images/'   
def downloadFromSrc(u):
    row = df.loc[u]
    imgClass = path+row['paintType'] + '/'
    url = row['src']
    filename = imgClass+str(u)+'.jpg'
    result = requests.get(url, stream=True)
    if result.status_code == 200:
        if not os.path.exists(imgClass):
            os.mkdir(imgClass)
        image = result.raw.read()
        #image = scipy.misc.imresize(arr=image,size=(128,128))
        open(filename,"wb").write(image)

pool = Pool()
pool.map(downloadFromSrc, range(len(df))) 

# Run this to resize all images to 128

In [1]:
from PIL import Image
import os, sys
from multiprocessing import Pool
path = '/home/jovyan/github/models/BAM-DCGAN/data/images/'
dirs = os.listdir( path )

def resize(path):
    print("Starting")
    dirs = os.listdir( path )
    for item in dirs:
        
        if os.path.isfile(path+item) and item!='procesed':
            try:
                im = Image.open(path+item)
                f, e = os.path.splitext(path+item)
                imResize = im.resize((128,128), Image.ANTIALIAS).convert('RGB')
                if not os.path.exists(path[:-1]+'_procesed/'):
                    os.makedirs(path[:-1]+'_procesed/')
                imResize.save(path[:-1]+'_procesed/'+item, 'JPEG', quality=100)
            except:
                pass
    print("DONE")

#resize()
ls = [path+o+'/' for o in os.listdir(path)]
pool = Pool()
pool.map(resize,ls)

Starting
Starting
Starting
Starting
Starting
Starting
Starting
Starting
DONE


/opt/conda/lib/python3.6/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


DONE
DONE
DONE
DONE
DONE
DONE
DONE


[None, None, None, None, None, None, None, None]